PyTorch Tutorial

In [1]:
# print progress, possibly faster than progress bar
def print_progress(epoch, batch_id, accuracy, loss):
    progress = '='* int((10. * batch_id / len(train_data)))
    progress += '>'
    if batch_id == 1:
        print('Train Epoche {}: {}% [{}]\t accuracy: {:.6f}, loss: {:.6f}'.format(
            epoch, 100. * batch_id / len(train_data),progress, accuracy, loss.item()), end='')
    else:
        print('\rTrain Epoche {}: {}% [{}]\t accuracy: {:.6f}, loss: {:.6f}'.format(
            epoch, 100. * batch_id / len(train_data),progress, accuracy, loss.item()), end='', flush = True)

Progress bar

In [2]:
from tqdm import trange
from time import sleep

def get_divisors(n):
    divisors = []
    for m in range(1, n+1):
        if n % m == 0:
            divisors.append(m)
    return divisors

iterations = 10
with trange(iterations, unit="carrots") as pbar:
    for i in pbar:
        sleep(0.5)
        if i % 2:
            pbar.set_description(f"Testing odd number {i}")
        else:
            pbar.set_description(f"Testing even number {i}")
        pbar.set_postfix(divisors=get_divisors(i))

Testing odd number 9: 100%|███████████████████████████████████| 10/10 [00:05<00:00,  1.97carrots/s, divisors=[1, 3, 9]]


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
from scipy import io
import os

# Tutorial: https://www.youtube.com/watch?v=GBzojftwfGQ&list=PLNmsVeXQZj7rx55Mai21reZtd_8m-qe27&index=5&ab_channel=TheMorpheusTutorials
# 1. Tensor
x = torch.Tensor(5,3)   # numpy arrays werden durch Tensoren ersetzt
x[:,:] = 1
y = torch.randn(5,3)    # random tenseor
#print(y)
xy = torch.add(x,y)
print(xy)

# 2. Für Grafik Karte
#if torch.cuda.is_available():
#    x = x.cuda()       # RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable 
#    y = y.cude()
#print(x+y)

# 3. Daten laden 
train_data = torch.utils.data.DataLoader(datasets.MNIST('data_figures_nn', train=True, download=True, 
                                                    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3082,))])),
                                    batch_size=64, shuffle=True, **kwargs)
# Bilder laden und verarbeiten
from PIL import Image
from torchvision import transforms
from os import listdir
normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])    # normalize around mean with sigma (std)
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(256), transforms.ToTensor(), normalize])
train_data_list = []
train_data = []
target_label = []
for f in listdir('catdog/train/'):
    img = Image.open('catdog/train/' + f)
    img_tensor = transforms(img)
    img_tensor.unsqueeze_(0)   # save directly in img_tensor with _ at the end (1,3,256,256)
    # label
    isCat = 1 if 'cat' in f else 0
    isDog = 1 if 'dog' in f else 0
    target = [isCat, isDog]
    target_label.append(target)
    if len(train_data_list) >= 64:
        train_data.append(torch.stack(train_data_list), target_label)
        train_data_list = []
        break




tensor([[ 0.0728,  1.8949, -0.6067],
        [ 3.9697, -0.7631,  1.1983],
        [ 0.8506,  0.9362,  0.3802],
        [ 0.0469,  0.2083, -0.1294],
        [ 0.2626,  0.8436,  0.6136]])


NameError: name 'datasets' is not defined

In [ ]:
# 4. Erste eigene Neuronale Netzwerk
class MyNet(nn.Module): # Vererbungslehre
    def __init__(self):              # initialize network
        super(MyNet,self).__init__()
        self.lin1 = nn.Linear(10,10)
        self.lin2 = nn.Linear(10,10)
    
    def forward(self, x):            # forward pass function
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

    def num_flat_features(self,x):
        size = x.size() [1:]         # ohne batch dimension
        num = 1
        for i in size:
            num *= i
        return num
    
net = MyNet()
#net = net.cuda()
print(net)

In [ ]:
# 5. main training loop with input
for i in range(100):
    x = [0,1,1,1,0,1,1,1,0,0]
    input = Variable(torch.Tensor([x for _ in range(10)]))    # 10x10 input variable, weil man ein batch von 10 daraus erstellt
    out = net(input)                      # input durch Netz durchlaufen lassen

    # labels what to expect
    x = [1,0,0,0,1,0,0,0,1,1]
    target = Variable(torch.Tensor([x for _ in range(10)]))
    # loss
    criterion = nn.MSELoss()              # Fehlerfunktionen: optimieren (softwmax, MSE, ...)
    loss = criterion(out, target)
    print(loss)

    net.zero_grad()
    loss.backward()
    optimizer = optim.SGD(net.parameters(), lr=0.1,)            # optimizer ausprobieren SGD, Adam, ...
    optimizer.step()
    y
    
test_input = Variable(torch.Tensor([1,1,1,1,0,0,0,1,1,1]))
test_out = net(test_input)
print(test_out)  

print(out)

# 6. Speichern und Laden des Trainierten Netzes (pro Zyklus speichern, falls das Programm abstürtzt während dem trainieren)
torch.save(net, 'data_figures_nn/MyNet.pt')
if os.path.isfile('data_figures_nn/MyNet.pt'):
    net_load = torch.load('data_figures_nn/MyNet.pt')

print(net_load)


MNIST example

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
import time
import os

kwargs = {}          # without GPU
#kwargs = {'num_workers': 1, 'pin_memory': True}        # with GPU
batch_size=64
train_data = torch.utils.data.DataLoader(datasets.MNIST('data_figures_nn', train=True, download=True, 
                                                    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3082,))])),
                                    batch_size=64, shuffle=True, **kwargs)    # shuffle that network don't learns repetition, kwargs are arguments for the function (has to be written)  
print(len(train_data.dataset))

test_data = torch.utils.data.DataLoader(datasets.MNIST('data_figures_nn', train=False, download=True, 
                                                    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3082,))])),
                                    batch_size=64, shuffle=True, **kwargs)    # shuffle that network don't learns repetition, kwargs are arguments for the function (has to be written)  

class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        # definition of object layers
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)    # (in_channels: Eingangsdimension, out_channels: Ausgangsdimension)
        # (kernel_size: Faltungsfiltergrösse, stride: #Schritte der Faltung (reduziert Dimension), padding: Leerer Rand dass Dimension weniger abnimmt, dilation: adding zeros between kernel, groups: , bias: added to output, padding_mode='zeros')
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        # (p=0.5: probability for dropout, inplace: if True input layer gets changed)
        self.conv_dropout = nn.Dropout2d()            # damit das Netz nichts auswendig lernt
        self.fc1 = nn.Linear(320, 60)
        self.fc2 = nn.Linear(60, 10)
    
    def forward(self,x):
        # actual layer composition
        print('start:',x.shape)
        x = self.conv1(x)
        print('conv1:', x.shape)
        # (kernel_size, stride, padding, dilation, return_indices: of max value if True, ceil_mode: ceil if True (aufrunden))
        x = F.max_pool2d(x, 2)
        print('pool1:', x.shape)
        x = F.relu(x)
        #print('relu1:',x.shape)
        x = self.conv2(x)
        #print('conv2:',x.shape)
        x = self.conv_dropout(x)
        #print('dropout:',x.shape)
        x = F.max_pool2d(x, 2)
        #print('pool2:',x.shape)
        x = F.relu(x)
        #print('relu2:', x.shape)
        x = x.view(-1,320)        # daten umformen
        #print('view:', x.shape)
        x = F.relu(self.fc1(x))
        #print('relu3_fc1:', x.shape)
        x = self.fc2(x)
        #print('fc2:', x.shape)
        
        return F.log_softmax(x, dim=1)
    
if os.path.isfile('data_figures_nn/MyNet_MNIST.pt'):
    net = torch.load('data_figures_nn/MyNet_MNIST.pt')       

# model
model = net()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.8)   # momentum [0,1]


# training
def train(epoch):
    model.train()
    #model.eval()
    with tqdm(train_data, unit="batch") as tepoch:
        # in our train data we have the pictures (data), target number 1-9 and bates of size 64 in this case
        for (data, target) in tepoch:
            #data = data.cuda()
            #target = target.cuda()
            #data = Variable(data)
            #target = Variable(target)
            optimizer.zero_grad()
            
            # forward pass
            out = model(data)
            
            #calculate loss
            loss = F.nll_loss(out, target)          # criterion = nn.CrossEntropyLoss() + loss = criterion(out, target), F.nll_loss is better
            predictions = out.argmax(dim=1, keepdim=True).squeeze()
            correct = (predictions == target).sum().item()
            accuracy = correct / batch_size
            
            loss.backward()
            optimizer.step()
            
            # display training progress
            #print_progress(epoch, batch_id, accuracy, loss)      # faster?
            tepoch.set_description(f"Epoch {epoch}")
            tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
            #time.sleep(0.1)
            torch.save(net, 'data_figures_nn/MyNet_MNIST.pt')

def test():
    model.eval()
    loss = 0
    correct = 0
    for data, target in test_data:
        out = model(data)
        loss += F.nll_loss(out, target, size_average=False).item()
        prediction = out.data.max(1, keepdim=True)[1]
        correct += prediction.eq(target.data.view_as(prediction)).sum()
    
    loss = loss / len(test_data.dataset)
    print('average loss: ', loss)
    print('accuracy: ', int(100.*correct/len(test_data.dataset)), '%')
        
    
for epoch in range(1,5):
        train(epoch)
        test()
        


60000


  0%|                                                                                       | 0/938 [00:00<?, ?batch/s]

start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])


Epoch 1:   0%|▏                                           | 4/938 [00:00<02:42,  5.74batch/s, accuracy=15.6, loss=2.27]

start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])


...

conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])


Epoch 4: 100%|████████████████████████████████████████| 938/938 [00:37<00:00, 24.97batch/s, accuracy=48.4, loss=0.0751]


start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([32, 1, 28, 28])
conv1: torch.Size([32, 10, 24, 24])
pool1: torch.Size([32, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
co

start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
co

conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([64, 1, 28, 28])
conv1: torch.Size([64, 10, 24, 24])
pool1: torch.Size([64, 10, 12, 12])
start: torch.Size([16, 1, 28, 28])
conv1: torch.Size([16, 10, 24, 24])
pool1: torch.Size([16, 10, 12, 12])
average loss:  0.08356346208453179
accuracy:  97 %
